# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f40b86e1ee0>
├── 'a' --> tensor([[-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294]])
└── 'x' --> <FastTreeValue 0x7f40b86e1fd0>
    └── 'c' --> tensor([[ 0.2123,  0.9684, -0.3233, -1.5284],
                        [-0.0228,  0.1428,  0.0943,  0.0094],
                        [-0.0640,  0.0836, -0.2168, -1.0332]])

In [4]:
t.a

tensor([[-0.3776,  0.8561, -0.6487],
        [ 0.4783, -0.8890, -1.1294]])

In [5]:
%timeit t.a

58.2 ns ± 0.0585 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f40b86e1ee0>
├── 'a' --> tensor([[ 1.1522, -0.6028,  0.0096],
│                   [ 0.4601, -0.2388,  0.4723]])
└── 'x' --> <FastTreeValue 0x7f40b86e1fd0>
    └── 'c' --> tensor([[ 0.2123,  0.9684, -0.3233, -1.5284],
                        [-0.0228,  0.1428,  0.0943,  0.0094],
                        [-0.0640,  0.0836, -0.2168, -1.0332]])

In [7]:
%timeit t.a = new_value

62.3 ns ± 0.0577 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294]]),
    x: Batch(
           c: tensor([[ 0.2123,  0.9684, -0.3233, -1.5284],
                      [-0.0228,  0.1428,  0.0943,  0.0094],
                      [-0.0640,  0.0836, -0.2168, -1.0332]]),
       ),
)

In [10]:
b.a

tensor([[-0.3776,  0.8561, -0.6487],
        [ 0.4783, -0.8890, -1.1294]])

In [11]:
%timeit b.a

59.1 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.6243, -1.0229,  1.5479],
               [ 0.0511,  0.3427, -1.1228]]),
    x: Batch(
           c: tensor([[ 0.2123,  0.9684, -0.3233, -1.5284],
                      [-0.0228,  0.1428,  0.0943,  0.0094],
                      [-0.0640,  0.0836, -0.2168, -1.0332]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

869 ns ± 0.326 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 622 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 355 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f40b874ef70>
├── 'a' --> tensor([[[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]],
│           
│                   [[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]],
│           
│                   [[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]],
│           
│                   [[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]],
│           
│                   [[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]],
│           
│                   [[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]],
│           
│                   [[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]],
│           
│                   [[-0.3776,  0.8561, -0.6487],
│                    [ 0.4783, -0.8890, -1.1294]]])
└── 'x' --> <FastTreeValue 0x7f400ca9fcd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 99.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4012fa7790>
├── 'a' --> tensor([[-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294],
│                   [-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294],
│                   [-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294],
│                   [-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294],
│                   [-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294],
│                   [-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294],
│                   [-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294],
│                   [-0.3776,  0.8561, -0.6487],
│                   [ 0.4783, -0.8890, -1.1294]])
└── 'x' --> <FastTreeValue 0x7f4017296340>
    └── 'c' --> tensor([[ 0.2123,  0.9684, -0.3233, -1.5284],
                        [-0.0228,  0.1428,  0.0943,  0.0094],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 689 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.7 µs ± 94.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]],
       
               [[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]],
       
               [[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]],
       
               [[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]],
       
               [[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]],
       
               [[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]],
       
               [[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]],
       
               [[-0.3776,  0.8561, -0.6487],
                [ 0.4783, -0.8890, -1.1294]]]),
    x: Batch(
           c: tensor([[[ 0.2123,  0.9684, -0.3233, -1.5284],
                       [-0.0228,  0.1428,  0.0943,  0.0094],
                       [-0.0640,  0.0836, -0.2168, -1.0332]],
         

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294],
               [-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294],
               [-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294],
               [-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294],
               [-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294],
               [-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294],
               [-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294],
               [-0.3776,  0.8561, -0.6487],
               [ 0.4783, -0.8890, -1.1294]]),
    x: Batch(
           c: tensor([[ 0.2123,  0.9684, -0.3233, -1.5284],
                      [-0.0228,  0.1428,  0.0943,  0.0094],
                      [-0.0640,  0.0836, -0.2168, -1.0332],
                      [ 0.2123,  0.9684, -0.3233, -1.5284],
                      [-0.0228,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

320 µs ± 704 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
